In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

2024-07-29 18:35:55.669419: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 18:35:55.788900: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 18:35:55.910041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 18:35:56.023301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 18:35:56.052563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 18:35:56.248691: I tensorflow/core/platform/cpu_feature_gu

In [2]:
train = pd.read_csv('data/train.csv', engine='pyarrow')
test = pd.read_csv('data/test.csv', engine='pyarrow')

In [3]:
train.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [4]:
col_drp = ['Unnamed: 0', 'id', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
train = train.drop(columns=col_drp)
test = test.drop(columns=col_drp)

In [5]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,Male,Loyal Customer,44,Business travel,Business,2,2,2,2,4,4,5,4,4,4,4,3,4,5,satisfied
1,Male,Loyal Customer,42,Business travel,Business,2,1,2,2,3,4,4,2,2,2,2,5,2,4,satisfied
2,Female,Loyal Customer,42,Business travel,Business,3,2,3,3,5,4,5,4,4,4,4,5,4,4,satisfied
3,Female,Loyal Customer,60,Business travel,Business,1,5,5,5,1,3,2,1,1,1,1,3,1,1,neutral or dissatisfied
4,Female,Loyal Customer,43,Business travel,Eco,5,1,1,1,1,1,5,3,3,5,3,4,3,5,satisfied


In [6]:
train['Gender'] = train['Gender'].apply(lambda x: 1 if x == 'Female' else 0)
test['Gender'] = test['Gender'].apply(lambda x: 1 if x == 'Female' else 0)

In [7]:
train['Customer Type'] = train['Customer Type'].apply(lambda x: 1 if x == 'Loyal Customer' else 0)
test['Customer Type'] = test['Customer Type'].apply(lambda x: 1 if x == 'Loyal Customer' else 0)

In [8]:
train['satisfaction'] = train['satisfaction'].apply(lambda x: 1 if x == 'satisfied' else 0)

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Class'] = le.fit_transform(train['Class'])
test['Class'] = le.transform(test['Class'])

In [10]:
train['Type of Travel'] = train['Type of Travel'].apply(lambda x: 1 if x == 'Business travel' else 0)
test['Type of Travel'] = test['Type of Travel'].apply(lambda x: 1 if x == 'Business travel' else 0)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train['Age'] = scaler.fit_transform(train[['Age']])
test['Age'] = scaler.transform(test[['Age']])

In [12]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,0,1,0.474359,1,0,2,2,2,2,4,4,5,4,4,4,4,3,4,5,1
1,0,1,0.448718,1,0,2,1,2,2,3,4,4,2,2,2,2,5,2,4,1
2,1,1,0.448718,1,0,3,2,3,3,5,4,5,4,4,4,4,5,4,4,1
3,1,1,0.679487,1,0,1,5,5,5,1,3,2,1,1,1,1,3,1,1,0
4,1,1,0.461538,1,1,5,1,1,1,1,1,5,3,3,5,3,4,3,5,1


In [13]:
test.head()

,Gender,Customer Type,Age,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness
0,0,1,0.371795,1,0,3,5,5,5,2,2,3,3,3,3,3,2,3,2
1,0,1,0.179487,0,1,4,4,4,3,2,4,2,2,4,5,4,5,4,2
2,1,0,0.269231,1,0,3,3,3,3,3,3,4,3,4,2,4,5,5,3
3,0,1,0.551282,1,0,5,5,5,5,5,2,2,4,4,4,4,5,4,4
4,0,1,0.679487,1,0,2,1,1,1,3,4,3,2,2,2,2,3,2,3


In [14]:
X = train.drop(columns='satisfaction')
y = train['satisfaction']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=.5)

In [163]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(X_train.shape[1], 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(150, activation='elu'),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(150, activation='elu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

In [164]:
initial_rate = .01
model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_rate),
             loss='binary_crossentropy',
             metrics=['accuracy'],
             )

In [ ]:
mcp = keras.callbacks.ModelCheckpoint('best_model_mcp.keras', save_best_only=True, monitor='val_accuracy', mode='max')

history = model.fit(X_train, y_train, 
          epochs=100, batch_size=64, 
          validation_data=[X_val, y_val], 
          callbacks=[mcp])

Epoch 1/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8444 - loss: 0.3884 - val_accuracy: 0.9326 - val_loss: 0.1717
Epoch 2/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9319 - loss: 0.1674 - val_accuracy: 0.9487 - val_loss: 0.1282
Epoch 3/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9363 - loss: 0.1554 - val_accuracy: 0.9386 - val_loss: 0.1567
Epoch 4/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9385 - loss: 0.1507 - val_accuracy: 0.9356 - val_loss: 0.1597
Epoch 5/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9410 - loss: 0.1415 - val_accuracy: 0.9447 - val_loss: 0.1338
Epoch 6/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9455 - loss: 0.1322 - val_accuracy: 0.9418 - val_loss: 0.1502
Epoch 7/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9468 - loss: 0.1283 - val_accuracy: 0.9454 - val_loss: 0.1253
Epoch 8/100
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9489 - loss: 0

In [160]:
model.evaluate(X_test, y_test)

325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.9509 - loss: 0.1105


[0.1167425587773323, 0.9487056136131287]

In [161]:
model_cp = keras.saving.load_model('best_model_mcp.keras')

In [162]:
model_cp.evaluate(X_test, y_test)

325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 928us/step - accuracy: 0.9582 - loss: 0.0998


[0.10930745303630829, 0.9541911482810974]

In [148]:
submission = model_cp.predict(test)
submission = np.where(submission >= .5, 'satisfied', 'neutral or dissatisfied')
submission = pd.DataFrame({'satisfaction': submission.flatten()})

812/812 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [149]:
submission.head()

,satisfaction
0,neutral or dissatisfied
1,neutral or dissatisfied
2,neutral or dissatisfied
3,satisfied
4,neutral or dissatisfied
